In [13]:
import numpy as np
import pandas as pd
import json
import matplotlib

In [14]:
with open('/home/kimino/soft/qcm-detector/results.json', 'r') as results_file:
    r_json = json.loads(results_file.read())

In [15]:
experiments = [r_json[exp] for exp in r_json]

In [16]:
def get_tf_positives(experiment, experiment_name):
    true_positives = experiment['True Positives']
    false_positives = [exp for exp in experiment['Detections'] if exp not in experiment['True Positives']]
    tp_df = pd.DataFrame(true_positives)
    fp_df = pd.DataFrame(false_positives)
    tp_df['is_correct'] = True
    fp_df['is_correct'] = False
    df = pd.concat([tp_df, fp_df], ignore_index= True)
    df['experiment'] = experiment_name
    df['path'] = experiment['Path']
    return df
def get_all_detections(experiment_json):
    detection_df = pd.DataFrame()
    for experiment in experiment_json:
        tf_df = get_tf_positives(experiment_json[experiment], experiment)
        detection_df = pd.concat([detection_df, tf_df], ignore_index= True, sort = False)
    return detection_df

In [17]:
results_df = get_all_detections(r_json)

ipykernel_launcher:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
results_df.groupby(['detector', 'is_correct']).count()

timestamp  frequency  resistance  period  time  \
detector        is_correct                                                   
Expo-predict    False               4          4           4       4     4   
                True               12         12          12      12    12   
IsolationForest False             446        446         446     446   446   
                True              458        458         458     458   458   
Mean-shift      True               25         25          25      25    25   
OneClassSVM     False              98         98          98      98    98   
                True               35         35          35      35    35   

                            severity  experiment  path  
detector        is_correct                              
Expo-predict    False              4           4     4  
                True              12          12    12  
IsolationForest False            446         446   446  
                True             458         458   458  
Mean-shift      True              25          25    25  
OneClassSVM     False             98          98    98  
                True              35          35    35

In [19]:
mean_shift_detection = results_df[results_df['detector'] == 'Mean-shift']

In [20]:
mean_shift_detection[results_df['is_correct'] == False]

ipykernel_launcher:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,timestamp,frequency,resistance,period,time,severity,detector,is_correct,experiment,path


In [24]:
times = np.sum([experiment['Total Detection Time'] for experiment in experiments])

In [25]:
times

6326.094820737839